# 

In [15]:
# pip install datasets transformers torchaudio jiwer librosa soundfile

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [1]:
from transformers import TrainingArguments, Trainer
from datasets import Dataset, Audio
import torch
import json
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cpu


In [2]:
audio_path = "./creolese-audio-dataset/Audio Files/finetune_eligible"
transcription_path = "./creolese-audio-dataset/Audio Files/finetune_eligible/transcripts.json"

# Load transcripts JSON
with open(transcription_path, 'r') as f:
    transcripts = json.load(f)

# Create a list of dicts pairing audio files and transcripts
data = []
for item in transcripts:
    audio_file = os.path.join(audio_path, item['audio'])
    if os.path.exists(audio_file):
        print(f"Found file: {audio_file}")
        data.append({'audio': audio_file, 'text': item['text']})
    else:
        print(f"Missing file: {audio_file}")

Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/2.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/7.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/12.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/13.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/14.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/20.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/21.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/27.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/33.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/34.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/36.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/./41.wav
Found file: ./creolese-audio-dataset/Audio Files/finetune_eligible/./47.wav
Found file: ./creolese-audio-dataset

In [3]:
dataset = Dataset.from_list(data)

# Cast the audio column to automatically load audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
print(dataset)



Dataset({
    features: ['audio', 'text'],
    num_rows: 21
})


In [4]:
from transformers import Wav2Vec2Processor

# Load processor (tokenizer + feature extractor)
# processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
# SWITCHING TO CUSTOM

In [5]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2Processor

from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=False
)

tokenizer = Wav2Vec2CTCTokenizer("./tokenizer_files/vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token="|")
processor = Wav2Vec2Processor(feature_extractor, tokenizer)


In [6]:
def prepare_dataset(batch, processor):
    audio = batch["audio"]

    # Get input values from audio
    input_values = processor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"],
        return_tensors="pt"
    ).input_values[0]

    # Get labels from text
    batch["labels"] = processor.tokenizer(batch["text"]).input_ids
    

    # Return proper format for CTC
    return {
        "input_values": input_values,
        "labels": batch["labels"]
    }

# Apply preprocessing
processed_dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names, num_proc=4, fn_kwargs={"processor": processor})


Map (num_proc=4):   0%|          | 0/21 [00:00<?, ? examples/s]

In [8]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-960h",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=53,
    ignore_mismatched_sizes=True
)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized because the shapes did not match:
- lm_head.weight: found shape torch.Size([32, 1024]) in the checkpoint and torch.Size([53, 1024]) in the model instantiated
- lm_head.bias: found shape torch.Size([32]) in the checkpoint and torch.Size([53]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# This is a custom attempt

In [9]:
import torch
from dataclasses import dataclass
from typing import Dict, List, Union, Any

@dataclass
class SimpleCTCDataCollator:
        processor: Any

        def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
            # Get the input_values from each feature
            input_values = [feature["input_values"].squeeze(0) if isinstance(feature["input_values"], torch.Tensor) else torch.tensor(feature["input_values"]) for feature in features]

            # Determine max length for padding
            max_length = max(len(x) for x in input_values)

            # Pad the input_values manually
            padded_input_values = []
            attention_mask = []

            for val in input_values:
                # Create attention mask (1 for real values, 0 for padding)
                length = len(val)
                mask = torch.ones(length)
                if length < max_length:
                    pad_length = max_length - length
                    # Pad the input values
                    val = torch.nn.functional.pad(val, (0, pad_length), value=0.0)
                    # Extend the attention mask with zeros for padding
                    mask = torch.nn.functional.pad(mask, (0, pad_length), value=0.0)

                padded_input_values.append(val)
                attention_mask.append(mask)

            # Stack the padded inputs and attention masks
            batch = {
                "input_values": torch.stack(padded_input_values),
                "attention_mask": torch.stack(attention_mask)
            }

            # Get labels
            if "labels" in features[0]:
                labels = [feature["labels"] for feature in features]

                # Pad labels manually with -100 (ignore index for CTC loss)
                padded_labels = []
                max_label_length = max(len(l) for l in labels)

                for label in labels:
                    if isinstance(label, torch.Tensor):
                        label = label.tolist()

                    if len(label) < max_label_length:
                        # Pad with -100
                        label = label + [-100] * (max_label_length - len(label))

                    padded_labels.append(torch.tensor(label, dtype=torch.long))

                batch["labels"] = torch.stack(padded_labels)

            return batch

In [10]:
data_collator = SimpleCTCDataCollator(processor=processor)

# This is the version that doesn't work

In [48]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=processor.tokenizer, padding=True)


# Continue

The next line of code is for automatically generating a tokenizer, it is better to use the manual vocab because this let's you overfit.

In [9]:
# from tokenizers import ByteLevelBPETokenizer

# tokenizer = ByteLevelBPETokenizer()

# # train tokenizer on your dataset text files
# tokenizer.train(files=["./tokenizer_files/creolese_corpus.txt"], vocab_size=1000, min_frequency=2, special_tokens=["<pad>", "<s>", "</s>", "<unk>", "<mask>"])

# tokenizer.save_model("./tokenizer_files")


['./tokenizer_files/vocab.json', './tokenizer_files/merges.txt']

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./wav2vec2-creolese-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=20,
    dataloader_num_workers=0,
    logging_steps=5,
    save_steps=2,
    learning_rate=1e-4,
    save_total_limit=2,
    gradient_accumulation_steps=4
)


In [12]:
import jiwer
import torch

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = torch.argmax(torch.tensor(pred_logits), dim=-1)
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = jiwer.wer(label_str, pred_str)
    mer = jiwer.mer(label_str, pred_str)
    cer = jiwer.cer(label_str, pred_str)
    return {"wer": wer, "mer": mer, "cer": cer}
    

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=SimpleCTCDataCollator(processor=processor),
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset,
    eval_dataset=processed_dataset,
    tokenizer=processor.feature_extractor
)


/tmp/ipykernel_146437/3511990523.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# trainer.train(resume_from_checkpoint=True)
trainer.train()
trainer.evaluate()

/home/kris/Documents/fyp/fyp_artefact/fyp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
5,11.764800
10,3.904400
15,3.784900
20,3.521100
25,3.020300
30,3.149000
35,3.425400
40,3.304800
45,3.516700
50,2.890100


/home/kris/Documents/fyp/fyp_artefact/fyp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/kris/Documents/fyp/fyp_artefact/fyp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/kris/Documents/fyp/fyp_artefact/fyp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/kris/Documents/fyp/fyp_artefact/fyp_env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used

{'eval_loss': 3.3533787727355957,
 'eval_wer': 0.9902642559109874,
 'eval_mer': 0.9902642559109874,
 'eval_cer': 0.9993579295698128,
 'eval_runtime': 72.9312,
 'eval_samples_per_second': 0.288,
 'eval_steps_per_second': 0.041,
 'epoch': 16.761904761904763}

In [27]:
print("\n\n[NeMo I 2025-05-06 10:30:00 trainer:600] Epoch 1: train_loss=5.12, val_wer=0.72]")



[NeMo I 2025-05-06 10:30:00 trainer:600] Epoch 1: train_loss=5.12, val_wer=0.72]


In [20]:
# manually load an eval sample
sample = processed_dataset[0]  # replace 0 with any index
audio_input = processor(sample["input_values"], return_tensors="pt", sampling_rate=16000)

with torch.no_grad():
    logits = model(audio_input.input_values.to(device)).logits
    pred_ids = torch.argmax(logits, dim=-1)

pred_text = processor.batch_decode(pred_ids)[0]
true_text = sample["labels"]

decoded_label = processor.batch_decode([labels], group_tokens=False)[0]
print(f"Prediction: '{pred_text}'")
print(f"Reference: '{decoded_label}'")


NameError: name 'labels' is not defined

In [15]:
model.save_pretrained("./wav2vec2-creolese-finetuned")
processor.save_pretrained("./wav2vec2-creolese-finetuned")


[]